In [ ]:
import inspect, os, sys

import re
import json
import statistics
import argparse
import itertools
from glob import glob
from pathlib import Path
from dataclasses import dataclass
from typing import  Iterable, Any


def cd_if_exists(dir: Path | str) -> bool:
    if isinstance(dir, str):
        dir = Path(dir)
    if dir.exists() and dir.is_dir():
        os.chdir(dir)
        return True
    else:
        return False

# Normally start VSCode (or whatever you use to run this notebook interactively) from the root, so
# we have to `cd` into the parent dir of this file, to access utils functions.
cd_if_exists('./scripts')

# These are util functions
from ldj import ldj
from utils import *

import numpy as np
import matplotlib.pyplot as plt
# from matplotlib.font_manager import FontProperties
import matplotlib.font_manager as fm
from matplotlib.patches import FancyBboxPatch
from matplotlib.ticker import FuncFormatter
from matplotlib.patches import PathPatch
from matplotlib.path import get_path_collection_extents
import seaborn as sns

from rich import print, pretty
from tabulate import tabulate
import pretty_errors
from catppuccin import PALETTE
from IPython.display import display, HTML

pretty.install()
print(f"{os.getcwd()=}")

EXPERIMENT_DIR = Path("../experiments/structured-junction-twoway")
assert EXPERIMENT_DIR.is_dir() and EXPERIMENT_DIR.exists()

flavor = PALETTE.latte.colors
# data = dict()


In [ ]:
# use LaTeX for text with matplotlib
sns.set_style("darkgrid")
# set background color of sns darkgrid to flavor.base.hex
plt.rcParams['axes.facecolor'] = flavor.base.hex
# set font color to flavor.text.hex
plt.rcParams['text.color'] = flavor.text.hex

font_dirs = ["./fonts/"]
# go through all fonts in the font directory and add them
for font_dir in font_dirs:
    for font in os.listdir(font_dir):
        fm.fontManager.addfont(f"{font_dir}/{font}")

prop_jbm = fm.FontProperties(fname='./fonts/JetBrainsMonoNerdFontMono-Regular.ttf')
prop = fm.FontProperties(fname='./fonts/STIXTwoText-VariableFont_wght.ttf')

plt.rcParams.update({
    # "text.usetex": True,
    "font.family": prop.get_name(),
    # "font.family": "stix",
    # "font.sans-serif": prop.get_name(),
    "mathtext.fontset": "stix",
    # "text.latex.preamble": r"\usepackage{fontenc}\usepackage{fontspec}\setmainfont{JetBrainsMonoNerdFontMono-Regular}",
})

print(prop.get_name())

colors = [
    (flavor.lavender.hex, 1.0, (1, 0), 'o'),
    (flavor.green.hex, 1.0, (2, 2), 'X'),
]

In [ ]:
@dataclass(frozen=True)
class Statistics:
    mean: float
    median: float
    stdev: float
    min: float
    max: float


    def display(self) -> None:
        data = [
            ["Mean", self.mean],
            ["Median", self.median],
            ["Standard Deviation", self.stdev],
            ["Min", self.min],
            ["Max", self.max]
        ]
        html_table = tabulate(data, headers=["Statistic", "Value"], tablefmt="html")
        centered_html_table = f"""
        <div style="display: flex; justify-content: center;">
            {html_table}
        </div>
        """
        display(HTML(html_table))
        # display(HTML(centered_html_table))


def compute_stats(data: list[float]) -> Statistics:
    return Statistics(
        mean=np.mean(data),
        median=np.median(data),
        stdev=np.std(data),
        min=np.min(data),
        max=np.max(data),
    )


In [ ]:
@dataclass(frozen=True)
class PerpendicularPositionErrorResult:
    errors: list[float]
    rmses: list[float]


def perpendicular_position_error(exported_data: dict) -> PerpendicularPositionErrorResult:
    errors: list[float] = []
    rmses: list[float] = []

    for robot_id, robot_data in exported_data['robots'].items():
        color: str = robot_data['color']
        positions = np.array([p for p in robot_data['positions']])
        mission = robot_data['mission']
        waypoints = []
        for route in mission['routes']:
            waypoints.append(route['waypoints'][0])
            for wp in route['waypoints'][1:]:
                waypoints.append(wp)

        waypoints = np.array(waypoints)
        waypoints = np.squeeze(waypoints)

        lines: list[LinePoints] = [LinePoints(start=start, end=end) for start, end in sliding_window(waypoints, 2)]
        closest_projections = [closest_projection_onto_line_segments(p, lines) for p in positions]

        error: float = np.sum(np.linalg.norm(positions - closest_projections, axis=1))
        rmse: float = np.sqrt(error / len(positions))

        errors.append(error)
        rmses.append(rmse)

    return PerpendicularPositionErrorResult(errors=errors, rmses=rmses)


In [ ]:
@dataclass(frozen=True)
class CollisionsResult:
    interrobot: int
    environment: int

def collisions(exported_data: dict) -> CollisionsResult:
    interrobot: int = len(exported_data['collisions']['robots'])
    environment: int = len(exported_data['collisions']['environment'])
    return CollisionsResult(interrobot=interrobot, environment=environment)


In [ ]:
@dataclass(frozen=True)
class TotalDistanceTravelledResult:
    distance: list[float]
    optimal_distance: list[float]

    @staticmethod
    def new() -> 'TotalDistanceTravelledResult':
        return TotalDistanceTravelledResult(distance=[], optimal_distance=[])

# def euclidian_distance(a: (float, float), b: (float, float)) -> float:

def total_distance_travelled(exported_data: dict) -> TotalDistanceTravelledResult:
    result = TotalDistanceTravelledResult.new()

    for robot_id, robot_data in exported_data['robots'].items():
        positions = robot_data['positions']

        for i in range(len(positions) - 1, -1, -1):
            point = positions[i]
            if abs(point[0]) > 50 or abs(point[1]) > 50:
                _ = positions.pop()

        positions = np.array([p for p in robot_data['positions']])

        waypoints = []
        mission = robot_data['mission']
        for route in mission['routes']:
            waypoints.append(route['waypoints'][0])
            for wp in route['waypoints'][1:]:
                waypoints.append(wp)

        waypoints = np.array(waypoints)
        waypoints = np.squeeze(waypoints)

        x =waypoints[-1][0]
        if abs(x) > 50:
            sign: int = -1 if x < 0.0 else 1
            x = sign * 50
            waypoints[-1][0] = x

        y =waypoints[-1][1]
        if abs(y) > 50:
            sign: int = -1 if y < 0.0 else 1
            y = sign * 50
            waypoints[-1][1] = y


        def accumulated_distance(points):
            # Compute pairwise Euclidean distances between successive points
            distances = np.sum(np.sqrt(np.sum(np.diff(points, axis=0)**2, axis=1)))
            return distances

        optimal_distance = accumulated_distance(waypoints)
        total_distance_travelled = accumulated_distance(positions)
        result.distance.append(total_distance_travelled)
        result.optimal_distance.append(optimal_distance)

    return result

In [ ]:
@dataclass(frozen=True)
class DurationResult:
    inner: list[float]


def durations(data_set: dict) -> DurationResult:
    iter = data_set['robots'].items()
    ds = np.empty(len(iter))
    
    for i, (_, robot_data) in enumerate(iter):
        m = robot_data['mission']
        finished_at: float = m['finished_at']
        started_at: float = m['started_at']
        dur: float = finished_at - started_at
        # print(f"{dur=}")
        ds[i] = dur

    return DurationResult(ds)

In [ ]:
@dataclass(frozen=True)
class Metric:
    ppe: PerpendicularPositionErrorResult
    collisions: CollisionsResult
    total_distance_travelled: TotalDistanceTravelledResult
    durations: DurationResult


In [ ]:
def flatten(lst: Iterable) -> list:
    return list(itertools.chain.from_iterable(lst))


In [ ]:
@dataclass
class Results:
    with_tracking: dict[str, Metric]
    without_tracking: dict[str, Metric]

results = Results(dict(), dict())

RE = re.compile(r"tracking-(on|off)-failure-(\d\.\d)\.json")

for f in EXPERIMENT_DIR.iterdir():
    print(f"{f=}")
    captures = RE.match(f.name)
    if captures is None:
        continue
    # assert captures is not None
    tracking = captures.group(1)
    failure_prob = captures.group(2)

    with open(f) as f:
        data = json.load(f)

    ppe = perpendicular_position_error(data)
    dist_travelled = total_distance_travelled(data)
    d = durations(data)
    metric = Metric(ppe=ppe, collisions=collisions(data), total_distance_travelled=dist_travelled, durations=d)
    

    match tracking:
        case "on":
            results.with_tracking[failure_prob] = metric
        case "off":
            results.without_tracking[failure_prob] = metric
        case _:
            assert False

In [ ]:
for (dataset, name) in [(results.with_tracking, "path"), (results.without_tracking, "waypoint")]:
    for p in range(8):
        p = str(p / 10.0)
        mean = np.mean(np.array(dataset[p].total_distance_travelled.distance) / np.array(dataset[p].total_distance_travelled.optimal_distance))
        print(f"{name} {p=} {mean=:.4f}")

# np.mean(np.array(results.without_tracking['0.0'].total_distance_travelled.distance) / np.array(results.without_tracking['0.0'].total_distance_travelled.optimal_distance))

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 8))


RE = re.compile(r"tracking-(on|off)-failure-(\d\.\d)\.json")

for f in EXPERIMENT_DIR.iterdir():
    captures = RE.match(f.name)
    assert captures is not None
    tracking = captures.group(1)
    failure_prob = captures.group(2)

    with open(f) as f:
        data = json.load(f)

for robot_id, robot_data in results.with_tracking['0.0']['robots'].items():

    
    print(f"{robot_id=}")

## Perpendicular Position Error

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 4))

with_tracking_data = {k: compute_stats(v.ppe.errors).mean for k, v in sorted(results.with_tracking.items())}
without_tracking_data = {k: compute_stats(v.ppe.errors).mean for k, v in sorted(results.without_tracking.items())}

#      linestyle='--',

plt.plot(
    [float(k) for k in with_tracking_data.keys()],
    with_tracking_data.values(),
    # label="With Tracking"
    # label="$\mathbf{With\ Tracking}$",
    label="Path Tracking",
    color=flavor.lavender.hex,
)

plt.plot(
    [float(k) for k in without_tracking_data.keys()],
    without_tracking_data.values(),
    # label="Without",
    label="Waypoint Tracking",
    color=flavor.green.hex,
)

legend = ax.legend(loc="upper left", fontsize=11)
# legend.get_frame().set_facecolor('#FF0000')  # Set the legend background color
legend.get_frame().set_edgecolor(flavor.overlay2.hex)  # Set the legend background color

# Define a formatter function
def percentage(x, pos):
    return f'{int(x * 100)}'

# Apply the formatter to the x-axis
ax.xaxis.set_major_formatter(FuncFormatter(percentage))

# Apply the custom font to x-ticks
for label in ax.get_xticklabels():
    label.set_fontproperties(prop_jbm)

for label in ax.get_yticklabels():
    label.set_fontproperties(prop_jbm)

label_args = dict(fontsize=12)

# ax.grid(True)
# ax.set_xlim(0, 0.8)
# ax.set_xlabel("Failure Probability $[\%]$", **label_args)
# # ax.set_xticks(np.arange(0, 0.8, 0.1))  # Setting xtick resolution to 0.1
# ax.set_ylabel("Total Collisions", **label_args)
# ax.set_title("Environment Collisions")
legend = ax.legend(loc="upper left", fontsize=11)
# legend.get_frame().set_facecolor('#FF0000')  # Set the legend background color

legend.get_frame().set_edgecolor(flavor.overlay2.hex)  # Set the legend background color Caption
ax.grid(True)
ax.set_xlim(0, 0.71)

# ax.set_xticks(np.arange(0, 1.1, 0.1))  # Setting xtick resolution to 0.1
ax.set_ylabel("Mean of Perpendicular Path Deviation $[m]$", **label_args)
ax.set_xlabel("Failure Probability $[\%]$", **label_args)

# plt.tight_layout()
# ax.set_title("Mean of Perpendicular Path Deviation")

plt.savefig("perpendicular-path-deviation-structured-junction-twoway.svg", pad_inches=1.0)
plt.savefig("perpendicular-path-deviation-structured-junction-twoway.pdf")

plt.show()

## Collisions

In [ ]:
fig, axs = plt.subplots(2, figsize=(8,8))

ax = axs[0]

with_tracking_data = {k: v.collisions.interrobot for k, v in sorted(results.with_tracking.items())}
without_tracking_data = {k: v.collisions.interrobot for k, v in sorted(results.without_tracking.items())}

ax.plot(
    [float(k) for k in with_tracking_data.keys()],
    with_tracking_data.values(),
    label="With Tracking"
)

ax.plot(
    [float(k) for k in without_tracking_data.keys()],
    without_tracking_data.values(),
    label="Without"
)

ax.grid(True)
ax.set_xlim(0, 1)
ax.set_xticks(np.arange(0, 1.1, 0.1))  # Setting xtick resolution to 0.1
ax.set_xlabel("failure prob.")
ax.set_ylabel("Interrobot Collisions")
plt.tight_layout()
ax.set_title("Interrobot Collisions")
ax.legend()

ax = axs[1]

with_tracking_data = {k: v.collisions.environment for k, v in sorted(results.with_tracking.items())}
without_tracking_data = {k: v.collisions.environment for k, v in sorted(results.without_tracking.items())}

ax.plot(
    [float(k) for k in with_tracking_data.keys()],
    with_tracking_data.values(),
    label="With Tracking"
)

ax.plot(
    [float(k) for k in without_tracking_data.keys()],
    without_tracking_data.values(),
    label="Without"
)

ax.grid(True)
ax.set_xlim(0, 1)
ax.set_xlabel("failure prob.")
ax.set_xticks(np.arange(0, 0.9, 0.1))  # Setting xtick resolution to 0.1
ax.set_ylabel("Environment Collisions")
ax.set_title("Environment Collisions")
ax.legend()



plt.tight_layout()
plt.savefig("collisions-structured-junction-twoway.svg")
plt.show()


In [ ]:
fix, ax = plt.subplots(1, figsize=(6,4))

# Interrobot
with_tracking_data = {k: v.collisions.interrobot for k, v in sorted(results.with_tracking.items())}
without_tracking_data = {k: v.collisions.interrobot for k, v in sorted(results.without_tracking.items())}


ax.plot(
    [float(k) for k in with_tracking_data.keys()],
    with_tracking_data.values(),
    # label="$\mathit{Interrobot}$       $\mathbf{With\ Tracking}$",
    label="$\mathit{Interrobot}$        Path Tracking",
    color=flavor.lavender.hex,
     linestyle='--',
)

ax.plot(
    [float(k) for k in without_tracking_data.keys()],
    without_tracking_data.values(),
    # label="$\mathit{Interrobot}$       Without",
    label="$\mathit{Interrobot}$        Waypoint Tracking",
    color=flavor.green.hex,
    linestyle='--',
)

# Environment
with_tracking_data = {k: v.collisions.environment for k, v in sorted(results.with_tracking.items())}
without_tracking_data = {k: v.collisions.environment for k, v in sorted(results.without_tracking.items())}

ax.plot(
    [float(k) for k in without_tracking_data.keys()],
    without_tracking_data.values(),
    # label="$\mathit{Environment}$   Without",
    label="$\mathit{Environment}$    Path Tracking",
    color=flavor.peach.hex
)

ax.plot(
    [float(k) for k in with_tracking_data.keys()],
    with_tracking_data.values(),
    # label="$\mathit{Environment}$   $\mathbf{With\ Tracking}$",
    label="$\mathit{Environment}$    Waypoint Tracking",
    color=flavor.red.hex
)

label_args = dict(fontsize=12)

ax.grid(True)
ax.set_xlim(0, 0.71)
ax.set_xlabel("Failure Probability $[\%]$", **label_args)
# ax.set_xticks(np.arange(0, 0.8, 0.1))  # Setting xtick resolution to 0.1
ax.set_ylabel("Total Collisions", **label_args)
# ax.set_title("Environment Collisions")
legend = ax.legend(loc="upper left", fontsize=11)
# legend.get_frame().set_facecolor('#FF0000')  # Set the legend background color
legend.get_frame().set_edgecolor(flavor.overlay2.hex)  # Set the legend background color

# Define a formatter function
def percentage(x, pos):
    return f'{int(x * 100)}'

# Apply the formatter to the x-axis
ax.xaxis.set_major_formatter(FuncFormatter(percentage))

# Apply the custom font to x-ticks
for label in ax.get_xticklabels():
    label.set_fontproperties(prop_jbm)

for label in ax.get_yticklabels():
    label.set_fontproperties(prop_jbm)

plt.yscale("linear")

# plt.tight_layout()
plt.savefig("collisions-structured-junction-twoway.svg")
plt.savefig("collisions-structured-junction-twoway.pdf")
plt.show()


In [ ]:
{k: v.collisions for k, v in sorted(results.with_tracking.items())}

# With Tracking

## Perpendicular Position Error

In [ ]:
# for failure_prob, metric in sorted(results.with_tracking.items()):
#     print(f"{failure_prob=}")
#     compute_stats(metric.ppe.errors).display()

## Collisions

In [ ]:
{k: v.collisions for k, v in sorted(results.with_tracking.items())}

## Total Distance Travelled

# Without Tracking

## Perpendicular Position Error

In [ ]:
# for failure_prob, metric in sorted(results.without_tracking.items()):
#     print(f"{failure_prob=}")
#     compute_stats(metric.ppe.errors).display()

## Collisions

In [ ]:
{k: v.collisions for k, v in sorted(results.without_tracking.items())}

In [ ]:
os.getcwd()


In [ ]:
import numpy as np
from scipy.spatial.distance import pdist, squareform


def pwdist(points: np.ndarray, distance_fn) -> np.ndarray:
    pass


# Example input array
# points = np.array([[x1, y1], [x2, y2], ..., [xn, yn]])
points = np.array([[0, 0], [1, 0], [1, 1], [1,2]])
dm = pdist(points, lambda u, v: np.sqrt(((u-v)**2).sum()))
print(f"{dm=}")


print(f"{pdist(points)=}")

print(f"{np.sum(pdist(points, 'euclidean'))=}")


# Compute pairwise Euclidean distances
# dist_matrix = squareform(pdist(points))

# Compute the accumulated distance
# accumulated_distance = np.sum(dist_matrix)
# print(f"{accumulated_distance=}")

In [ ]:
def accumulated_distances(points):
    # Compute pairwise Euclidean distances between successive points
    distances = np.sum(np.sqrt(np.sum(np.diff(points, axis=0)**2, axis=1)))
    return distances

# Example usage
points = np.array([[0, 0], [1, 1], [2,2]])
accumulated_distances(points)

In [ ]:
np.abs([-60, 52])

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 4))

with_tracking_data = {k: compute_stats(v.durations.inner).mean for k, v in sorted(results.with_tracking.items())}
without_tracking_data = {k: compute_stats(v.durations.inner).mean for k, v in sorted(results.without_tracking.items())}

#      linestyle='--',

plt.plot(
    [float(k) for k in with_tracking_data.keys()],
    with_tracking_data.values(),
    # label="With Tracking"
    # label="$\mathbf{With\ Tracking}$",
    label="Path Tracking",
    color=flavor.lavender.hex,
)

plt.plot(
    [float(k) for k in without_tracking_data.keys()],
    without_tracking_data.values(),
    # label="Without",
    label="Waypoint Tracking",
    color=flavor.green.hex,
)

legend = ax.legend(loc="upper left", fontsize=11)
# legend.get_frame().set_facecolor('#FF0000')  # Set the legend background color
legend.get_frame().set_edgecolor(flavor.overlay2.hex)  # Set the legend background color

# Define a formatter function
def percentage(x, pos):
    return f'{int(x * 100)}'

# Apply the formatter to the x-axis
ax.xaxis.set_major_formatter(FuncFormatter(percentage))

# Apply the custom font to x-ticks
for label in ax.get_xticklabels():
    label.set_fontproperties(prop_jbm)

for label in ax.get_yticklabels():
    label.set_fontproperties(prop_jbm)

label_args = dict(fontsize=12)

# ax.grid(True)
# ax.set_xlim(0, 0.8)
# ax.set_xlabel("Failure Probability $[\%]$", **label_args)
# # ax.set_xticks(np.arange(0, 0.8, 0.1))  # Setting xtick resolution to 0.1
# ax.set_ylabel("Total Collisions", **label_args)
# ax.set_title("Environment Collisions")
legend = ax.legend(loc="upper left", fontsize=11)
# legend.get_frame().set_facecolor('#FF0000')  # Set the legend background color

legend.get_frame().set_edgecolor(flavor.overlay2.hex)  # Set the legend background color Caption
ax.grid(True)
ax.set_xlim(0, 0.71)

# ax.set_xticks(np.arange(0, 1.1, 0.1))  # Setting xtick resolution to 0.1
ax.set_ylabel("Mission Duration $[s]$", **label_args)
ax.set_xlabel("Failure Probability $[\%]$", **label_args)

# plt.tight_layout()
# ax.set_title("Mean of Perpendicular Path Deviation")

# plt.savefig("perpendicular-path-deviation-structured-junction-twoway.svg", pad_inches=1.0)
# plt.savefig("perpendicular-path-deviation-structured-junction-twoway.pdf")

plt.show()